In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import dlib
import uuid
import tensorflow as tf

In [2]:
detector = dlib.get_frontal_face_detector()

In [15]:
# PATH = os.path.join('D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\database\\Test\\Kamya')
PATH = os.path.join('D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\database\\Train\\Varsha')
PATH

'D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\database\\Train\\Varsha'

In [46]:
def increase_brightness(img, value=30):
    # img = os.path.join(PATH,im)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [53]:
im = cv2.imread('database\\Train\\Shrey\\fb91105e-2af8-11ed-a74f-a42b5c8c19cf.jpg')
# img = increase_brightness(im,500)
# cv2.imshow("img",img)
# cv2.waitKey()
# cv2.destroyAllWindows()
type(np.array(im))
# type(im)

numpy.ndarray

In [63]:
cap = cv2.VideoCapture(0)
i = 0
while i<15:
    ret, frame = cap.read()
    if ret:
        results = detector(frame,1)
        if (len(results)>0):
            x1 = results[0].left()
            y1 = results[0].top()
            x2 = results[0].right()
            y2 = results[0].bottom()
            
            if(cv2.waitKey(1) & 0XFF == ord('a')):
                img_crop = frame[y1:y2-100,x1:x2]
                # img_crop = cv2.resize(img_crop,(200,200))
                cv2.imshow("crop",img_crop)
                img_name = os.path.join(PATH,'{}.jpg'.format(uuid.uuid1()))
                i = i+1
                cv2.imwrite(img_name, img_crop)
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2, cv2.LINE_AA)
            cv2.rectangle(frame, (x1,y1), (x2,y2-100), (0,0,0), 2, cv2.LINE_AA)
        cv2.imshow("image",frame)
        if(cv2.waitKey(1)==27):
            break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
def data_aug(img):
    data = []
    for i in range(4):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
            
        data.append(img)
    
    return data

In [18]:
for file_name in os.listdir(os.path.join(PATH)):
    img_path = os.path.join(PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())